## Import Python libraries

In [80]:
!pip install flwr

In [81]:
import tensorflow.compat.v1 as tf
sess = tf.Session()
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import os
print(os.listdir("../input"))
import errno
import tensorflow as tf
import csv
from keras import backend as K
import sklearn
from tensorflow import keras
from keras import Sequential
from keras.layers import Flatten, Dense, Dropout, BatchNormalization
from keras.layers import Conv1D, MaxPool1D
from keras.optimizers import Adam
import random
import pandas as pd
import numpy as np
import math
import os
import flwr as fl
import array
from scipy.stats import dirichlet
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

from imblearn.over_sampling import SMOTE

['adult-census-income']


In [82]:
!pip install aif360 

In [83]:
!pip install BlackBoxAuditing

## Import dataset

In [84]:
path = ('/kaggle/input/adult-census-income/adult.csv')
df = pd.read_csv(path, encoding='latin-1')

In [85]:
df.race.unique()

array(['White', 'Black', 'Asian-Pac-Islander', 'Other',
       'Amer-Indian-Eskimo'], dtype=object)

## Exploratory Data Analysis

### Check shape of dataset

In [86]:
df.shape

(32561, 15)

We can see that there are 32561 instances and 15 attributes in the data set.

### Preview dataset

In [87]:
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [88]:
df.race.value_counts()

White                 27816
Black                  3124
Asian-Pac-Islander     1039
Amer-Indian-Eskimo      311
Other                   271
Name: race, dtype: int64

### View summary of dataframe

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education.num   32561 non-null  int64 
 5   marital.status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital.gain    32561 non-null  int64 
 11  capital.loss    32561 non-null  int64 
 12  hours.per.week  32561 non-null  int64 
 13  native.country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


### Encode `?` as `NaNs`

In [90]:
df[df == '?'] = np.nan

In [91]:
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,NaN,77053,HS-grad,9,Widowed,NaN,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,NaN,186061,Some-college,10,Widowed,NaN,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [92]:
df.isna().sum()

age                  0
workclass         1836
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     583
income               0
dtype: int64

### Impute missing values with mode

In [93]:
for col in ['workclass', 'occupation', 'native.country']:
    df[col].fillna(df[col].mode()[0], inplace=True)

In [94]:
df.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
income            0
dtype: int64

In [95]:
X = df.drop(['income'], axis=1)
y = df['income']

In [96]:
y.unique(),y.nunique()

(array(['<=50K', '>50K'], dtype=object), 2)

In [97]:
X.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,90,Private,77053,HS-grad,9,Widowed,Prof-specialty,Not-in-family,White,Female,0,4356,40,United-States
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States
2,66,Private,186061,Some-college,10,Widowed,Prof-specialty,Unmarried,Black,Female,0,4356,40,United-States
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States


## Split data into separate training and test set

In [98]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [99]:
from sklearn import preprocessing

categorical = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country']
for feature in categorical:
        le = preprocessing.LabelEncoder()
        X_train[feature] = le.fit_transform(X_train[feature])
        X_test[feature] = le.transform(X_test[feature])

In [100]:
X = pd.concat([X_train, X_test], axis=0)
y = pd.concat([y_train, y_test], axis=0)
data = pd.concat([X, y], axis=1)
data=data.reset_index(drop=True)

In [101]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

## Feature Scaling

In [102]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X.columns)

X_test = pd.DataFrame(scaler.transform(X_test), columns = X.columns)

In [103]:
X_train.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,0.101484,2.600478,-1.494279,-0.332263,1.133894,-0.402341,-0.782234,2.214196,0.39298,-1.430470,-0.145189,-0.217407,-1.662414,0.262317
1,0.028248,-1.884720,0.438778,0.184396,-0.423425,-0.402341,-0.026696,-0.899410,0.39298,0.699071,-0.145189,-0.217407,-0.200753,0.262317
2,0.247956,-0.090641,0.045292,1.217715,-0.034095,0.926666,-0.782234,-0.276689,0.39298,-1.430470,-0.145189,-0.217407,-0.038346,0.262317
3,-0.850587,-1.884720,0.793152,0.184396,-0.423425,0.926666,-0.530388,0.968753,0.39298,0.699071,-0.145189,-0.217407,-0.038346,0.262317
4,-0.044989,-2.781760,-0.853275,0.442726,1.523223,-0.402341,-0.782234,-0.899410,0.39298,0.699071,-0.145189,-0.217407,-0.038346,0.262317


# Rejection option-based classification (ROC)

Rejection option-based classification (ROC) is a method of classification that allows the classifier to abstain from making a prediction when it is not confident in its decision. This can be done by assigning a probability to each class, and then rejecting any instance with a probability below a certain threshold.

ROC can be used to improve the accuracy and fairness of classification models. By rejecting instances where the model is not confident, ROC can reduce the number of misclassifications, and can also help to mitigate the effects of bias in the data.

There are several different ways to implement ROC. One common approach is to use a Bayesian classifier, which assigns a probability to each class for each instance. The classifier can then be configured to reject any instance with a probability below a certain threshold.

Another approach to ROC is to use a decision tree. Decision trees can be used to classify instances by recursively splitting the data into smaller and smaller subsets. At each split, the tree chooses the feature that best distinguishes between the two classes. If the tree is not able to make a confident decision, it can reject the instance.

ROC can be a useful tool for improving the accuracy and fairness of classification models. However, it is important to note that ROC can also reduce the number of instances that are classified, which can impact the utility of the model.

Here are some of the advantages of using ROC:

Can improve the accuracy of classification models by reducing the number of misclassifications.
Can help to mitigate the effects of bias in the data.
Can be used with a variety of different classification algorithms.
Here are some of the disadvantages of using ROC:

Can reduce the number of instances that are classified.
Can be more complex to implement than traditional classification methods.
Can require more data to train the model.

In [104]:
y_probs.shape

(19538,)

In [105]:
y_rejected.shape

(9769,)

In [106]:
from sklearn.linear_model import LogisticRegression
# Train a classification model
model = LogisticRegression()
model.fit(X_train, y_train)

# Calculate the probabilities of each instance being classified as positive
y_probs = model.predict_proba(X_test)

# Set a threshold for rejection
threshold = 0.5

# Reject any instances with a probability below the threshold
y_rejected = y_probs[:, 1] < threshold

# Evaluate the accuracy and fairness of the model on the testing set
accuracy = (y_test == y_probs.argmax(axis=1)).mean()
fairness = (y_test[y_rejected] == y_probs[y_rejected].argmax(axis=1)).mean()

print('Accuracy:', accuracy)
print('Fairness:', fairness)

Accuracy: 0.8216808271061521
Fairness: 0.8411991331567542


****

In [107]:
y_pred_roc = model_roc.predict(X_test)
yprod = model_roc.predict_proba(X_test)

**Comparing Results**

In [108]:
y_test.shape

(9769,)

In [109]:
y_pred_no_roc.shape

(9769,)

In [110]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
# Train the model without ROC
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict the class labels on the testing set without ROC
y_pred_no_roc = model.predict(X_test)

# Evaluate the model without ROC
accuracy_no_roc = accuracy_score(y_test, y_pred_no_roc)
precision_no_roc = precision_score(y_test, y_pred_no_roc)
recall_no_roc = recall_score(y_test, y_pred_no_roc)
f1_no_roc = f1_score(y_test, y_pred_no_roc)
y_test = y_test.reshape(-1)
auc_no_roc = roc_auc_score(y_test, y_pred_no_roc)

# Train the model with ROC
model_roc = LogisticRegression(class_weight='balanced')
model_roc.fit(X_train, y_train)

# Predict the class labels on the testing set with ROC
y_pred_roc = model_roc.predict(X_test)

# Evaluate the model with ROC
accuracy_roc = accuracy_score(y_test, y_pred_roc)
precision_roc = precision_score(y_test, y_pred_roc)
recall_roc = recall_score(y_test, y_pred_roc)
f1_roc = f1_score(y_test, y_pred_roc)
auc_roc = roc_auc_score(y_test, y_pred_roc)

print('Accuracy without ROC:', accuracy_no_roc)
print('Precision without ROC:', precision_no_roc)
print('Recall without ROC:', recall_no_roc)
print('F1 without ROC:', f1_no_roc)
print('AUC without ROC:', auc_no_roc)

print('Accuracy with ROC:', accuracy_roc)
print('Precision with ROC:', precision_roc)
print('Recall with ROC:', recall_roc)
print('F1 with ROC:', f1_roc)
print('AUC with ROC:', auc_roc)

Accuracy without ROC: 0.8216808271061521
Precision without ROC: 0.7108680792891319
Recall without ROC: 0.44086477320898687
F1 without ROC: 0.54421768707483
AUC without ROC: 0.691889876483036
Accuracy with ROC: 0.7720339850547651
Precision with ROC: 0.5193434935521688
Recall with ROC: 0.7511657481983891
F1 with ROC: 0.6141050077976087
AUC with ROC: 0.7649216055431891
